In [256]:
import numpy as np
import math as m
import itertools
import scipy as sp

def binary_decode(bytearray: list[int]):
    bytearray = np.flip(bytearray)
    return sum(bytearray * (2 ** np.array(list(enumerate(bytearray.flat)))[:,0]))

def binary_encode(ingest: list[int], num_bits: int):
    return np.floor_divide(ingest, (2 ** np.array(list(enumerate(range(0,(num_bits)))))[:,0])) % 2

def func(i):
    return m.tanh(i)

def float_func(i, precision, ancillary):
    num_bits = precision+ancillary
    val = i * (2**(precision))
    #print(val)
    bin_val = binary_encode(val,num_bits)
    return np.array(np.flip(bin_val))

def float_decomp(i, precision, ancillary):
    num_bits = precision+ancillary
    exponent = np.array([0]*ancillary + list(range(1,precision+1)))
    div = np.array([2]*(num_bits))
    div = div ** exponent
    div = np.array([1]*(num_bits)) / div
    return np.sum(div*i)

res = func(0.999)

print(res)

encode = float_func(res,6,2)

float_decomp(encode,6,2)

0.76117386166057


0.75

In [257]:

def build_unitary(precision, ancillary):

    num_bits = precision+ancillary

    bin_inputs = np.array(list(itertools.product([0, 1], repeat=(precision))))

    bin_inputs = np.unique(np.pad(bin_inputs,(ancillary,0)), axis=0)

    inputs = np.apply_along_axis(float_decomp, 1, bin_inputs,precision,ancillary)

    index_inputs =  np.array(list(enumerate(inputs.tolist())))[:,0].astype(int)

    outputs = np.array([func(i) for i in inputs])

    bin_outputs = np.array([float_func(o,precision,ancillary) for o in outputs])

    index_outputs = np.array([binary_decode(bo) for bo in bin_outputs]).astype(int)

    output_vectors_matrix = sp.sparse.csr_matrix((np.ones(index_outputs.shape),(index_inputs,index_outputs)),(int(2**num_bits),(2**num_bits))).toarray()

    return np.linalg.qr(output_vectors_matrix)[0]

precision = 8

ancillary = 0

num_bits = precision + ancillary

unitary = build_unitary(precision,ancillary)

print(unitary)

[[-0.70710678  0.          0.         ...  0.04419417  0.04419417
   0.04419417]
 [-0.70710678  0.          0.         ... -0.04419417 -0.04419417
  -0.04419417]
 [-0.         -1.          0.         ...  0.          0.
   0.        ]
 ...
 [-0.         -0.         -0.         ...  0.5         0.
   0.        ]
 [-0.         -0.         -0.         ...  0.          0.5
  -0.5       ]
 [-0.         -0.         -0.         ...  0.         -0.5
   0.5       ]]


In [258]:
def apply_unitary(unitary, dec_input, precision, ancillary):

    num_bits = precision + ancillary

    input = float_func(dec_input, precision, ancillary)

    input_basis_state = np.zeros(2**num_bits)

    input_basis_state[int(binary_decode(input))] = 1

    output_basis_state = np.abs(input_basis_state @ unitary)

    max_probs = np.argwhere(output_basis_state == np.amax(output_basis_state))

    top_predictions = np.array([float_decomp(np.flip(binary_encode(max_prob,num_bits)),precision,ancillary) for max_prob in max_probs])

    return top_predictions[0]


In [259]:


num_tests = 15

test_inputs = np.random.rand(num_tests)

predictions = np.array([apply_unitary(unitary,predict,precision,ancillary) for predict in test_inputs])

print(predictions)

actuals = np.array([func(real) for real in (test_inputs)])

print(actuals)

np.average(predictions - actuals)

[0.67578125 0.1171875  0.54296875 0.5703125  0.171875   0.21484375
 0.125      0.22265625 0.515625   0.1484375  0.53515625 0.0078125
 0.109375   0.73046875 0.73046875]
[0.6791216  0.12429147 0.54448887 0.57412012 0.17710745 0.21612021
 0.13084101 0.22710978 0.52103551 0.15383879 0.54038013 0.01331355
 0.11284968 0.73565355 0.73393659]


-0.004415970442021451

In [260]:
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit.library import UnitaryGate

qr = QuantumRegister(num_bits)

qc = QuantumCircuit(qr)

u = UnitaryGate(unitary)

qc.append(u,qr)

qc.draw()

┌──────────┐
q43_0: ┤0         ├
       │          │
q43_1: ┤1         ├
       │          │
q43_2: ┤2         ├
       │          │
q43_3: ┤3         ├
       │  Unitary │
q43_4: ┤4         ├
       │          │
q43_5: ┤5         ├
       │          │
q43_6: ┤6         ├
       │          │
q43_7: ┤7         ├
       └──────────┘